# Wikification

In [104]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

from PyPDF2 import PdfReader
import os
import pandas as pd
import numpy as np

import sys
sys.path.append('../src/')
import wikifier

Number of processors:  8


In [ ]:
path = "../dat/books/all_books/"
data = pd.DataFrame(columns = ["file","text"])

book_files = os.listdir(path)

for j, book in enumerate(book_files):
    reader = PdfReader(path + book)
    print("Extracting " + book)
    text = ""
    k = 0
    for page in reader.pages:
        k+=1
        if k%50==0: print("Extract page" + str(k))
        text += str(page.extract_text()) + " "

    data = pd.concat([data, pd.DataFrame.from_records([{'file': book, 'text': text}])])
data.reset_index()

In [116]:
data.text.apply(len)

# exclude Math14, Strang

file
CollegeAlgCoreq-WEB.pdf                         1742146
Linear algebra — Strang.pdf                     1151894
linear-Cherey, Denton.pdf                        485450
ElementaryAlgebra2e-WEB_EjIP4sI.pdf             1628431
interactive_textbook.pdf                         578111
textbook_Hoffman_Kunze.pdf                       790385
Nicholson-OpenLAWA-2019A.pdf                    1402815
Linear algebra done right — Axler.pdf            520380
Math1410_print.pdf                               740115
Hefferon_LinAlgebra.pdf                          876866
Kuttler-LinearAlgebra-AFirstCourse-2017A.pdf     929580
Beezer_First_Course.pdf                         1077274
Name: text, dtype: int64

In [ ]:
data.text[8]

In [ ]:
batch_size = 5000
num_threads = mp.cpu_count()

all_annotations = []
for i in range(2,3):
    text = data.iloc[i].text

    pool = mp.Pool(num_threads)
    print("Length: " + str(len(text)))
    annotations = [pool.apply(wikifier.get_annotations, args=[text[i*batch_size:(i+1)*batch_size], i*batch_size])
        for i in range(int(np.floor(len(text)/batch_size)))]
    pool.close()
    all_annotations = sum(annotations, [])

In [ ]:
annos = [{
    "title": a["title"],
    "supports": [{'chFrom': s['chFrom'], 'chTo': s['chTo']} for s in a["support"] if s['pageRank']>0.001]
    } for a in all_annotations]
[a for a in annos if len(a['supports'])]

In [129]:
len(all_annotations)
import pickle
file = open("../dat/annotations_Denton.pkl", 'wb')
pickle.dump(all_annotations, file)
file.close()

In [19]:
import pickle
file = open("../dat/annotations/Beezer_First_Course.pkl", 'rb')
annos = pickle.load(file)

/var/folders/ly/txhzmb053hl2qybdkm0f018h0000gn/T/ipykernel_31929/1788237135.py:3: RuntimeWarning: coroutine 'main' was never awaited
  annos = pickle.load(file)


In [20]:
len(annos)

46129